In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.plscore_RMT import PLScoreRMT
from lib.models.LeeNet.score_pureRMT_mlp import ScorePureRMTMLP
from lib.models.LeeNet.score_pureRMT_center import ScorePureRMTCENTER
from lib.models.head.mlp import MLP
from lib.models.head.center_predictor import CenterPredictor
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PLScoreRMT(cfg=cfg)
    
    stride = cfg.model.backbone.stride
    feat_sz = int(cfg.data.search.size / stride)
    head = CenterPredictor(inplanes=cfg.model.pureRMT.embed_dim[-1], channel=cfg.model.head.num_channels, feat_sz=feat_sz, stride=stride)
    model = ScorePureRMTCENTER(backbone, head, cfg)
    return model


cfg = env_setting(cfg_name='plscore_pureRMT_center00001.yaml')
loader_train, loader_val = build_dataloaders(cfg)
data = None
# for i, _data in enumerate(loader_train, 1):
#     data = _data
#     break

In [5]:
# net = build_model(cfg)
# 
# focal_loss = FocalLoss()
# objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
# loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
# actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

In [6]:

# data = data.to(cfg.train.device)
# actor(data)

gt_gaussian_maps shape:  torch.Size([128, 1, 20, 20])
pred_dict['score_map'] shape torch.Size([128, 1, 20, 20])


(tensor(970.3591, device='cuda:3', grad_fn=<AddBackward0>),
 {'Loss/total': 970.359130859375,
  'Loss/giou': 1.6835126876831055,
  'Loss/l1': 0.866348147392273,
  'Loss/location': 960.2088623046875,
  'IoU': 0.0035139115061610937})

In [5]:
checkpoint_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/LeeNet_plScore_RMT_MLP00001/ScorePureRMTMLP_ep0740.pth.tar"
checkpoint_dict = torch.load(checkpoint_path,map_location={"cuda:2":"cuda:3"})['net']

net = build_model(cfg)

model_dict = net.state_dict()
state_dict = {k: v for k, v in checkpoint_dict.items() if k in model_dict.keys()}
print(state_dict.keys())
model_dict.update(state_dict)
net.load_state_dict(model_dict)

dict_keys(['backbone.patch_embed.proj.weight', 'backbone.patch_embed.proj.bias', 'backbone.score.conv1.conv.weight', 'backbone.score.conv1.bn.weight', 'backbone.score.conv1.bn.bias', 'backbone.score.conv1.bn.running_mean', 'backbone.score.conv1.bn.running_var', 'backbone.score.conv1.bn.num_batches_tracked', 'backbone.score.conv2.conv.weight', 'backbone.score.conv2.bn.weight', 'backbone.score.conv2.bn.bias', 'backbone.score.conv2.bn.running_mean', 'backbone.score.conv2.bn.running_var', 'backbone.score.conv2.bn.num_batches_tracked', 'backbone.score.confident_conv1.conv.weight', 'backbone.score.confident_conv1.bn.weight', 'backbone.score.confident_conv1.bn.bias', 'backbone.score.confident_conv1.bn.running_mean', 'backbone.score.confident_conv1.bn.running_var', 'backbone.score.confident_conv1.bn.num_batches_tracked', 'backbone.score.confident_conv2.conv.weight', 'backbone.score.confident_conv2.bn.weight', 'backbone.score.confident_conv2.bn.bias', 'backbone.score.confident_conv2.bn.running_

<All keys matched successfully>

In [6]:


focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

trainer.train(cfg.train.epoch)

gt_gaussian_maps shape:  torch.Size([128, 1, 20, 20])
pred_dict['score_map'] shape torch.Size([128, 1, 20, 20])
Epoch Time: 0:00:58.330300
Avg Data Time: 56.51694
Avg GPU Trans Time: 0.14170
Avg Forward Time: 1.67166
gt_gaussian_maps shape:  torch.Size([128, 1, 20, 20])
pred_dict['score_map'] shape torch.Size([128, 1, 20, 20])
Epoch Time: 0:00:59.102715
Avg Data Time: 28.29221
Avg GPU Trans Time: 0.11776
Avg Forward Time: 1.14139
gt_gaussian_maps shape:  torch.Size([128, 1, 20, 20])
pred_dict['score_map'] shape torch.Size([128, 1, 20, 20])
Epoch Time: 0:00:59.735261
Avg Data Time: 18.87865
Avg GPU Trans Time: 0.11367
Avg Forward Time: 0.91943
gt_gaussian_maps shape:  torch.Size([128, 1, 20, 20])
pred_dict['score_map'] shape torch.Size([128, 1, 20, 20])
Epoch Time: 0:01:00.366571
Avg Data Time: 14.17660
Avg GPU Trans Time: 0.10701
Avg Forward Time: 0.80803
gt_gaussian_maps shape:  torch.Size([128, 1, 20, 20])
pred_dict['score_map'] shape torch.Size([128, 1, 20, 20])
Epoch Time: 0:01:00.

KeyboardInterrupt: 